# License

In [ ]:
# Copyright 2021 University of San Andres' Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2021 University of San Andres
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Clasificación Fashion MNIST

En esta primera instancia, vamos a declarar o importar las dependencias que necesitamos para comenzar a trabajar con el tutorial.

A través de las palabras reservadas de Python3, como por ejemplo `import`, inicializaremos los siguientes paquetes.

In [ ]:
%load_ext tensorboard

import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

Luego, verificamos que tengamos correctamente instalada la placa de video, o mejor dicho, la *Graphical Processing Unit* (*GPU*).

In [ ]:
gpu_devices = tf.config.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

print('Num GPU available:', len(gpu_devices))

## Descargá el set de datos Fashion MNIST

Este tutorial usa el conjunto de datos más famoso conocido como [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist). Éste contiene 70.000 imágenes de distintas prendas de vestir que pertenecen a alguna de las 10 categorías. Cada imagen son de baja resolución (28x28 pixels) como se ve aquí:

<table>
  <tr><td align="center">
    <img src="https://github.com/zalandoresearch/fashion-mnist/raw/master/doc/img/fashion-mnist-sprite.png"
         alt="Fashion MNIST Sprite" width="1024">
  </td></tr>
  <tr><td align="center">
    <b>Figura 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Muestras Fashion MNIST</a><br/>&nbsp;
  </td></tr>
</table>

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

Al ejecutar el código de la celda anterior, devuelven los datos en las variables:

- `train_images` y `train_labels` que se utilizarán en el momento del entrenamiento de la red neuronal.
- `test_images` y `test_labels` serán utilizadas a la hora de evaluar el model entrenado.

Las etiquetas recolectadas por lo creadores de los datos corresponden a la siguiente tabla:

| Label | Class       |
|-------|-------------|
| 0     | T-shirt/top |
| 1     | Trouser     |
| 2     | Pullover    |
| 3     | Dress       |
| 4     | Coat        |
| 5     | Sandal      |
| 6     | Shirt       |
| 7     | Sneaker     |
| 8     | Bag         |
| 9     | Ankle boot  |

Las imágenes son vectores Numpy de 28x28, con valores de píxeles desde 0 hasta 255. Las etiquetas son vectores de enteros, desde el 0 al 9.

In [ ]:
class_names = [
    'T-shirt/top', 
    'Trouser', 
    'Pullover', 
    'Dress', 
    'Coat',
    'Sandal', 
    'Shirt',
    'Sneaker', 
    'Bag', 
    'Ankle boot'
]

## Explorando los datos

Es muy importante saber con qué datos estamos trabajando. Esto significa saber qué tamaño tienen los datos, las dimensiones de las imágenes, así como también saber cuántos datos tenemos para entrenar, y cuántos datos tenemos para hacer las pruebas de la red, una vez entrenada.

Lo primero es visualizar el tamaño del set de entrenamiento. Para eso ejecutamos la siguiente celda.

In [ ]:
train_images.shape

Vemos que existen 60000 imágenes, donde cada imágen tiene 28 de ancho y 28 de alto, tal y como habíamos dicho en un principio.

Luego, si tenemos la cantidad de etiquetas suficiente para las imágenes.

In [ ]:
len(train_labels)

Y vemos que efectivamente tenemos una etiqueta por cada imagen.

Del mismo modo hacemos para el set de prueba.

In [ ]:
test_images.shape

Tenemos 10.000 imágenes, donde cada una es de 28x28.

In [ ]:
len(test_labels)

Y también tenemos la cantidad de etiquetas correcta para este otro set de datos.

## Preprocesando los datos

Al igual que muchos otros set de datos, es importante aplicar transformaciones o un preprocesamiento de tal modo que la red neuronal tenga menos trabajo por hacer y le sea más fácil.

In [ ]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

Se ve que la imagen tiene valores en sus pixels que van desde 0 a 255. Una gran ayuda para la red neuronal es normalizar los valores entre 0 y 1. Para ello dividimos cada uno de los pixels por `255.0`.

In [ ]:
train_images = train_images / 255.0

test_images = test_images / 255.

# Show the figures
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## Construyendo el modelo

Lo siguiente, y una de las partes claves del entrenamiento es definir la arquitectura de la red neuronal. Para eso Tensorflow nos ayudará para que sea fácil describir tal arquitecutra.

Utilizaremos una arquitectura de profundidad 3, es decir, que tiene 3 capas. En este caso podemos usar las siguientes:

* `tf.keras.layers.Flatten(input_shape=(28, 28))`
* `tf.keras.layers.Dense(128, activation='relu')`
* `tf.keras.layers.Dense(10)`

A la primer capa le vamos a poner `input_shape` de tal modo que sepamos el tamaño que haya que aplicarle a la entrada de la red. En este caso, serán imágenes de `28x28`. Las otras dos capas van a ser de tipo `Dense`, es decir, donde a cada una le especificamos la cantidad de neuronas que van a tener. La segunda es de 128 y la última de 10.

**IMPORTANTE**: Pueden probar otro tipo de capas como las siguientes si se animan. Sino pueden ver otro de nuestros tutoriales con estas capas. 

Para más capas seguir este link: https://www.tensorflow.org/api_docs/python/tf/keras/layers

In [ ]:
model = tf.keras.Sequential([
    # Complete the layers
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Entrenamiento

Para entrenar, primero vamos a inicializar Tensorboard para que nos muestre el entrenamiento de una manera más entrenida. Y luego, ponemos a entrenar nuestra red neuronal.

In [ ]:
import os
import datetime

basedir = '/tmp/fashion/'
logdir = os.path.join(basedir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
os.makedirs(logdir, exist_ok=True)

In [ ]:
%tensorboard --reload_multifile True --logdir {basedir}

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
model.fit(train_images, train_labels, epochs=1, callbacks=[tensorboard_callback])

### Evaluar precisión

Una vez entrenado, hay que ver cómo se porta con información o imágenes nunca antes vistas. ¡Veamos!

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(f'\nTest accuracy: {test_acc * 100:.2f}%')

### Hacer predicciones

También podemos hacer predicciones e imprimir exactamente lo que está intentando clasificar la red. Para eso vamos a generar un modelo nuevo con un capa adicional al final llamada `Softmax`. Y al ejecutar la siguiente celda, vamos a ver que la red neuronal está diciendo que para una imagen, existe posibilidades de que sea una de las opciones. Pero les asigna un valor (una probabilidad) a cada clase de tal modo que el valor más grande es con el que nos vamos a quedar.

In [ ]:
probability_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax(),
])

predictions = probability_model.predict(test_images)

predictions[0]

En la siguiente celda nos quedamos con la predicción más grande.

In [ ]:
np.argmax(predictions[0])

Y verificamos que la etiqueta verdadera, ¡también es un 9!

In [ ]:
test_labels[0]

### Visualizar y verificar más predicciones

Ahora vamos verificar las predicciones de una manera más visual.

Por favor, ejecuten la siguiente celda para seguir avanzando.


In [ ]:
def plot_image(i, predictions_array, true_label, img):
  true_label, img = true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  true_label = true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

Veamos la imagen, la predicción y la probabilidad de clasificación, todo en uno.

In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [ ]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()

### Usando el modelo entrenado

Elijan la foto que más les guste y verifiquen si la red se equivoca o no.

In [ ]:
# Grab an image from the test dataset by modifiying the number in test_images
img = test_images[1]

plt.figure()
plt.imshow(img)
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
# Add the image to a batch where it's the only member.
img = (np.expand_dims(img,0))
print(img.shape)

Ejecuten la siguiente celda para ver si está en lo correcto.

In [ ]:
predictions_single = probability_model.predict(img)

plot_value_array(1, predictions_single[0], test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)
plt.show()